In [ ]:
Analysis of 

In [1]:
import re
import sys
import math
import pandas as pd
from collections import Counter

sys.path.append("../")

from utils.helpers import extract_values
from utils.evaluation_data import process_dataset, sample_texts

In [2]:
df = pd.read_json("../dataset/blip_laion_cc_sbu_558k.json")

In [3]:
df.head()

,id,image,conversations
0,4539375,00453/004539375.jpg,"[{'from': 'human', 'value': 'Render a clear an..."
1,2239345,00223/002239345.jpg,"[{'from': 'human', 'value': 'Write a terse but..."
2,5947502,00594/005947502.jpg,"[{'from': 'human', 'value': '<image> What is t..."
3,5116462,00511/005116462.jpg,"[{'from': 'human', 'value': '<image> Render a ..."
4,2017886,00201/002017886.jpg,"[{'from': 'human', 'value': 'What is in the ph..."


In [4]:
df[['human_value', 'gpt_value']] = df['conversations'].apply(extract_values)

In [5]:
df.head()

,id,image,conversations,human_value,gpt_value
0,4539375,00453/004539375.jpg,"[{'from': 'human', 'value': 'Render a clear an...",Render a clear and concise summary of the phot...,select luxury furniture 3 - inch gel memory fo...
1,2239345,00223/002239345.jpg,"[{'from': 'human', 'value': 'Write a terse but...",Write a terse but informative summary of the p...,a grey watch with an army style strap
2,5947502,00594/005947502.jpg,"[{'from': 'human', 'value': '<image> What is t...",<image>\nWhat is this?,a dragon kite flying in the blue sky stock images
3,5116462,00511/005116462.jpg,"[{'from': 'human', 'value': '<image> Render a ...",<image>\nRender a clear and concise summary of...,$ 10 - cute cheap printed mini dress - khaki m...
4,2017886,00201/002017886.jpg,"[{'from': 'human', 'value': 'What is in the ph...",What is in the photo?\n<image>,augmented reality using aruco markers in opencv


In [6]:
# Process the dataset
processed_df = process_dataset(df, 'gpt_value')

In [7]:
def calculate_adaptive_threshold(word_count):
    min_threshold = 0.3
    max_threshold = 0.8
    decay_factor = 0.05
    
    threshold = max_threshold * math.exp(-decay_factor * word_count) + min_threshold
    return min(max(threshold, min_threshold), max_threshold)

def filter_repeated_words(text):
    # Split the text into words
    words = re.findall(r'\w+', str(text).lower())
    
    if not words:
        return False  # Empty text
    
    # Special case for two-word sentences
    if len(words) == 2:
        return words[0] == words[1]
    
    # Count the occurrences of each word
    word_counts = Counter(words)
    
    # Calculate the adaptive threshold
    threshold = calculate_adaptive_threshold(len(words))
    
    # Calculate the proportion of the most common word
    most_common_word_count = word_counts.most_common(1)[0][1]
    proportion = most_common_word_count / len(words)
    
    # Return True if the proportion is above the threshold
    return proportion > threshold

def check_repeated_words(df, text_column, new_column_name='has_repeated_words'):
    # Apply the filter_repeated_words function to the text column
    df[new_column_name] = df[text_column].apply(filter_repeated_words)
    return df

In [8]:
# 191 Repeated Words
repeated_words_df = check_repeated_words(df, 'gpt_value')

In [9]:
filtered_df = repeated_words_df[~repeated_words_df['has_repeated_words']]

In [14]:
# Sample texts
sampled_texts = sample_texts(filtered_df, 'gpt_value', n_samples=30, n_bins=5)

# Display the results
pd.set_option('display.max_colwidth', None)  # To show full text content

D:\Github\maya-dataset-creation\notebooks\..\utils\evaluation_data.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['complexity_bin'] = create_bins(df, 'complexity', n_bins, use_qcut)
D:\Github\maya-dataset-creation\notebooks\..\utils\evaluation_data.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['length_bin'] = create_bins(df, 'length', n_bins, use_qcut)
D:\Github\maya-dataset-creation\notebooks\..\utils\evaluation_data.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy o

                                                                                                gpt_value  \
233410                                                              large stone birdbath with birds on it   
8687                                                                       an old brass kettle on a stove   
322946                                                          led switch light with touch button on off   
413038                                                                     the giants team on fire trucks   
479777                                             lawn lawn die set and die set hedgehogs from lawn fawn   
170303                                                                         logitech usb gamepad, blue   
173767                                                                  the pink flaming party bar set up   
111005                                                       we r onyx heavyweight hoodie navy size large   
352961             

In [19]:
sampled_texts[['gpt_value']].to_csv('../dataset/prompt_engineering.csv', index=False)